This example demonstrates how to use the 1D Heat Exchanger model from IDAES PSE with a single property package, specifically the IAPWS Steam Table. In this example, one stream will be heated while the other will be cooled.

The simulation of the 1D Heat Exchanger involves fixing any 7 of the following degrees of freedom:

- Two of the shell parameters: length, diameter, or area.
- Two of the tube parameters: length, diameter, or area.
- Number of tubes.
- Wall temperature (at all spatial points).
- Heat transfer coefficient (at all spatial points, for both shell and tube).


In [1]:
# Import pyomo package
from pyomo.environ import ConcreteModel, Constraint, value, units

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import the IAPWS property package to create a properties block for steam in the flowsheet
from idaes.models.properties import iapws95

from idaes.models.properties.iapws95 import htpx

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
from idaes.core.util.model_statistics import degrees_of_freedom

# Import the default IPOPT solver
from idaes.core.solvers import get_solver

# Import a heat exchanger unit
from idaes.models.unit_models.heat_exchanger_1D import (HeatExchanger1D,
                                                        HeatExchangerFlowPattern)

# Import a feed and product stream
from idaes.models.unit_models import Feed, Product

## Setting up the flowsheet

Creating a `concretemodel` foundation, attach the steady state flowsheet and declaring the property package on the shell and tube side of the heat exchanger

In [2]:
m = ConcreteModel()

m.fs = FlowsheetBlock(dynamic=False)

m.fs.properties_shell = iapws95.Iapws95ParameterBlock()

m.fs.properties_tube = iapws95.Iapws95ParameterBlock()

m.fs.properties = iapws95.Iapws95ParameterBlock()

## Defining Unit Operation Ports

In this section, we define the key inlet and outlet ports for our 1D heat exchanger model:

* **Cooling Water Inlet** – represents the feed stream of cooling water entering the heat exchanger.
* **Cooling Water Outlet** – represents the exit stream of cooling water after it has absorbed heat.
* **Steam Inlet** – represents the incoming hot steam that needs to be condensed.
* **Condensate Outlet** – represents the outlet stream of condensed steam (liquid water) after heat transfer.


In [3]:
m.fs.cooling_water_inlet = Feed(property_package=m.fs.properties)
m.fs.cooling_water_outlet = Product(property_package=m.fs.properties)
m.fs.steam_inlet = Feed(property_package=m.fs.properties)
m.fs.condensate_outlet = Product(property_package=m.fs.properties)
m.fs.heatexchanger = HeatExchanger1D(
    hot_side_name = 'shell',
    cold_side_name = 'tube',
    shell={"property_package": m.fs.properties_shell,
           "transformation_method": "dae.finite_difference",
           "transformation_scheme": "BACKWARD"},
    tube={"property_package": m.fs.properties_tube,
          "transformation_method": "dae.finite_difference",
          "transformation_scheme": "BACKWARD"},
    finite_elements=20,
    flow_type=HeatExchangerFlowPattern.cocurrent
)


In [4]:
# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

The initial DOF is 29


## Connecting the unit opearations

In [5]:
# Import necessary Pyomo components
from pyomo.environ import TransformationFactory

# Import the Arc component for connecting unit models
from pyomo.network import Arc

In [6]:
m.fs.s01 = Arc(source=m.fs.cooling_water_inlet.outlet, destination=m.fs.heatexchanger.cold_side_inlet)
m.fs.s02 = Arc(source=m.fs.steam_inlet.outlet, destination=m.fs.heatexchanger.hot_side_inlet)
m.fs.s03 = Arc(source=m.fs.heatexchanger.cold_side_outlet, destination=m.fs.cooling_water_outlet.inlet)
m.fs.s04 = Arc(source=m.fs.heatexchanger.hot_side_outlet, destination=m.fs.condensate_outlet.inlet)
TransformationFactory("network.expand_arcs").apply_to(m)

# Call the degrees_of_freedom function, get initial DOF
degrees_of_freedom(m)

29

## Setting up the conditions

In [7]:
m.fs.cooling_water_inlet.flow_mol[0].fix(1.0)
m.fs.cooling_water_inlet.pressure[0].fix(101325)
m.fs.cooling_water_inlet.enth_mol[0].fix(htpx(300*units.K, P = 101325*units.Pa))

m.fs.steam_inlet.flow_mol[0].fix(1.0)
m.fs.steam_inlet.pressure[0].fix(101325)
m.fs.steam_inlet.enth_mol[0].fix(htpx(400*units.K, P = 101325*units.Pa))

m.fs.heatexchanger.area.fix(0.5) # m2
m.fs.heatexchanger.length.fix(4.85) # m
m.fs.heatexchanger.heat_transfer_coefficient.fix(500) # W/m2/K

degrees_of_freedom(m)

0

In [8]:
# Initialize the flowsheet, and set the output at INFO
m.fs.cooling_water_inlet.initialize(outlvl=idaeslog.INFO)
m.fs.cooling_water_outlet.initialize(outlvl=idaeslog.INFO)
m.fs.steam_inlet.initialize(outlvl=idaeslog.INFO)
m.fs.condensate_outlet.initialize(outlvl=idaeslog.INFO)
m.fs.heatexchanger.initialize(outlvl=idaeslog.INFO)

2025-05-11 21:30:10 [INFO] idaes.init.fs.cooling_water_inlet: Initialization Complete.
2025-05-11 21:30:10 [INFO] idaes.init.fs.cooling_water_outlet: Initialization Complete.
2025-05-11 21:30:10 [INFO] idaes.init.fs.steam_inlet: Initialization Complete.
2025-05-11 21:30:10 [INFO] idaes.init.fs.condensate_outlet: Initialization Complete.
2025-05-11 21:30:10 [INFO] idaes.init.fs.heatexchanger.hot_side: Initialization Complete
2025-05-11 21:30:10 [INFO] idaes.init.fs.heatexchanger.cold_side: Initialization Complete
2025-05-11 21:30:11 [INFO] idaes.init.fs.heatexchanger: Initialization Complete.


In [9]:
# Solve the simulation using ipopt
# Note: If the degrees of freedom = 0, we have a square problem
opt = get_solver()
solve_status = opt.solve(m, tee = True)

Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=200


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes fo

In [10]:
# Display a readable report
m.fs.heatexchanger.report()


Unit : fs.heatexchanger                                                    Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key    : Value   : Units      : Fixed : Bounds
      Area : 0.50000 : meter ** 2 :  True : (None, None)
    Length :  4.8500 :      meter :  True : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                         Units         shell Inlet  shell Outlet  tube Inlet  tube Outlet
    Molar Flow          mole / second      1.0000        1.0000       1.0000      1.0000 
    Mass Flow       kilogram / second    0.018015      0.018015     0.018015    0.018015 
    T                          kelvin      400.00        373.12       300.00      369.73 
    P                          pascal  1.0132e+05    1.0132e+05   1.0132e+05  1.0132e+05 
    Vapor Fraction      dimensionless      1.0000       0.89483       0.

In [11]:
m.fs.visualize("Heat Exchanger 1D Model")

2025-05-11 21:31:04 [INFO] idaes.idaes_ui.fv.fsvis: Started visualization server
2025-05-11 21:31:04 [INFO] idaes.idaes_ui.fv.fsvis: Saving flowsheet to default file 'Heat Exchanger 1D Model.json' in current directory (/home/viraj/Documents/Github/IDAES PSE Examples/Heat_Exchanger_1D)
2025-05-11 21:31:04 [WARNING] idaes.idaes_ui.fv.fsvis: Flowsheet name changed: old='Heat Exchanger 1D Model' new='Heat-Exchanger-1D-Model'
Flowsheet name changed to 'Heat-Exchanger-1D-Model'
2025-05-11 21:31:04 [INFO] idaes.idaes_ui.fv.fsvis: Flowsheet visualization at: http://localhost:55279/app?id=Heat-Exchanger-1D-Model


VisualizeResult(store=<idaes_ui.fv.persist.FileDataStore object at 0x70bed65c9a30>, port=55279, server=<idaes_ui.fv.model_server.FlowsheetServer object at 0x70bed67bc140>, save_diagram=<bound method SaveDiagramScreenshot.save_diagram_screenshot of <idaes_ui.fv.save_diagram_screenshot.SaveDiagramScreenshot object at 0x70bed5807290>>)